# 记录已完成的编号

In [4]:
import os

def extract_folder_ids(source_dir, output_file):
    """
    遍历指定目录，提取所有直接子文件夹的名称（ID）并保存到文本文件
    
    参数:
    source_dir (str): 源文件夹路径
    output_file (str): 输出文本文件路径
    """
    try:
        # 检查源文件夹是否存在
        if not os.path.exists(source_dir):
            raise FileNotFoundError(f"源文件夹不存在: {source_dir}")
        
        # 获取所有直接子文件夹的名称
        folder_ids = [name for name in os.listdir(source_dir) 
                      if os.path.isdir(os.path.join(source_dir, name))]
        
        # 确保输出文件的目录存在
        output_dir = os.path.dirname(output_file)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # 将ID写入文本文件
        with open(output_file, 'w') as f:
            for folder_id in folder_ids:
                f.write(f"{folder_id}\n")
        
        print(f"成功提取 {len(folder_ids)} 个ID到文件: {output_file}")
        
    except Exception as e:
        print(f"处理过程中发生错误: {e}")

if __name__ == "__main__":
    # 源文件夹路径
    ## source_directory = "/data_lg/keru/DP_conclusion/DP_gpt_250502/generated_images"
    source_directory = "/data_lg/keru/DP_conclusion/DP_gpt_250617/generated_images"
    
    # 输出文件路径（可自定义）
    output_txt_file = "/data_lg/keru/DP_conclusion/txt/250617.txt"
    
    # 调用函数执行提取
    extract_folder_ids(source_directory, output_txt_file)    

成功提取 4897 个ID到文件: /data_lg/keru/DP_conclusion/txt/250617.txt


In [ ]:
import base64
import base64
import requests
import os
import json
import openai
import re
from tqdm import tqdm
from collections import defaultdict

# API配置
base_url = 
api_key = 
model = "sora_image"  # Use your actual model name here, e.g., "gpt-4o-image-vip"
client = openai.OpenAI(
    base_url=base_url,
    api_key=api_key,
)


# 路径配置
folder_path = "/data_lg/keru/DP_conclusion/DP_gpt_250502/valid_result"
output_json_path = "/data_lg/keru/DP_conclusion/DP_gpt_250617/output_log.json"
output_image_folder = "/data_lg/keru/DP_conclusion/DP_gpt_250617/generated_images"
error_txt_path = "/data_lg/keru/DP_conclusion/txt/result_error_2.txt"

# ID文本文件路径
valid_ids_path = "/data_lg/keru/DP_conclusion/txt/valid.txt"
existing_ids_1 = "/data_lg/keru/DP_conclusion/txt/250608.txt"
existing_ids_2 = "/data_lg/keru/DP_conclusion/txt/250502.txt"
new_ids_output_path = "/data_lg/keru/DP_conclusion/txt/250617.txt"

os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(os.path.dirname(new_ids_output_path), exist_ok=True)

# 提示词配置
prompt_groups = [
    "这是一件上装的前后，请参照为我重新生成上装的前后面,并将结果拼接呈现 ,高清,还原输入的衣服上所有花纹（碎花、文字、图案等）、颜色、样式。假如输入的衣服上面没有花纹，请不要额外添加！严格按照输入图的样式！背景为白色,细节清晰,图片规格,高1024,宽768*2（上装前面宽为768，上装后面为768），生成结果尽量保持居中，衣服长宽比需合理。后背中央空缺部分可能是头发遮挡，可按照空缺部分最近的颜色进行填充，生成完整衣服。一定要注意前后图案的生成要准确，尤其是后背！！！若有帽子、花纹、图案等务必还原！",
    "这是一件下装的前后，请参照为我重新生成下装的前后面 并将结果拼接呈现 ,高清,还原花纹（碎花、文字、图案等）、颜色、样式。假如输入的衣服上面没有花纹，请不要额外添加！严格按照输入图的样式！背景为白色,细节清晰,图片规格,高1024,宽768*2（上装前面宽为768，上装后面为768），生成结果尽量保持居中，衣服长宽比需合理。请注意，下装的前后长度需保持一致且合理，不会出现三角裤的情况，请避免内裤类型的生成,下装不包括鞋子和袜子！！裤子请按合理比例生成，避免长裤变成七分裤的情况。生成结果尽量真实",
    "这是一件套装的前后，套装可能是裙子，背带裤，请将他看作一件整体的衣服。请参照输入为我重新生成套装的前后面 并将结果拼接呈现 ,高清,还原花纹（碎花、文字、图案等）、颜色、样式。假如输入的衣服上面没有花纹，请不要额外添加！严格按照输入图的样式！背景为白色,细节清晰,图片规格,高1024,宽768*2（上装前面宽为768，上装后面为768），生成结果尽量保持居中，衣服长宽比需合理。后背中央空缺部分可能是头发遮挡，可按照空缺部分最近的颜色进行填充，生成完整衣服，生成结果尽量真实"
]

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def download_image(image_url, save_path):
    try:
        image_url = image_url.strip().rstrip(')')
        url_match = re.search(r'(https?://[^\s]+\.(?:png|jpg|jpeg|gif|bmp|webp))', image_url, re.IGNORECASE)
        if not url_match:
            print(f"Invalid URL format: {image_url}")
            return
        image_url = url_match.group(1)
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(image_url, headers=headers, timeout=15)
        if response.status_code == 200:
            with open(save_path, 'wb') as img_file:
                img_file.write(response.content)
            print(f"Image saved to {save_path}")
        else:
            print(f"Download failed ({response.status_code}): {image_url}")
    except Exception as e:
        print(f"Download error: {e}")

def generate_and_save_image(image_paths, folder_name, output_json_path, prompt, model, client, output_image_folder, pair_index, image_files, new_ids_set, overall_progress):
    base64_images = [encode_image(image_path) for image_path in image_paths]
    numbers = [re.search(r'part_(\d+)', image_file).group(1) for image_file in image_files]
    image_filename = f"result_{numbers[0]}_{numbers[1]}_generated.jpg"
    save_image_path = os.path.join(output_image_folder, folder_name, image_filename)
    os.makedirs(os.path.dirname(save_image_path), exist_ok=True)

    configure = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_images[0]}"}}
            ]}
        ]
    }

    response = client.chat.completions.create(**configure)
    data = response.choices[0].message.content
    download_image(data, save_image_path)

    output_log = {"image_paths": image_paths, "response": data}
    with open(output_json_path, 'a') as json_file:
        json.dump(output_log, json_file, indent=4)
        json_file.write("\n")
    
    new_ids_set.add(folder_name)
    overall_progress.update(1)  # 更新整体进度

def load_ids_from_txt(file_path):
    if not os.path.exists(file_path):
        print(f"文件不存在: {file_path}，返回空集")
        return set()
    with open(file_path, 'r') as f:
        return {line.strip() for line in f if line.strip()}

def load_error_ids(error_txt_path):
    if not os.path.exists(error_txt_path):
        print(f"错误文件不存在: {error_txt_path}，返回空集")
        return set()
    with open(error_txt_path, "r") as f:
        return {line.strip() for line in f if line.strip()}

def save_new_ids_to_txt(new_ids_set, output_path):
    with open(output_path, 'w') as f:
        for folder_id in sorted(new_ids_set):
            f.write(f"{folder_id}\n")
    print(f"已将 {len(new_ids_set)} 个新ID保存到: {output_path}")

def process_images_from_folder(folder_path, output_json_path, prompt_groups, model, client, output_image_folder):
    valid_ids = load_ids_from_txt(valid_ids_path)
    existing_ids = load_ids_from_txt(existing_ids_1) | load_ids_from_txt(existing_ids_2)
    error_ids = load_error_ids(error_txt_path)
    
    final_subjects = (valid_ids - existing_ids) | error_ids
    all_input_ids = {f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))}
    final_subjects = final_subjects & all_input_ids
    
    total_pairs = sum(
        (len([f for f in os.listdir(os.path.join(folder_path, f)) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]) + 1) // 2
        for f in final_subjects
    )
    
    print(f"总共有 {len(final_subjects)} 个ID，预计处理 {total_pairs} 组图像对")
    
    new_ids_set = set()
    # 创建整体进度条
    with tqdm(total=total_pairs, desc="整体进度", unit="组") as overall_progress:
        for folder_name in sorted(final_subjects):
            subject_folder_path = os.path.join(folder_path, folder_name)
            print(f"\n处理文件夹 {folder_name}")
            image_files = [f for f in os.listdir(subject_folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            image_files.sort(key=lambda x: int(re.search(r'part_(\d+)', x).group(1)))

            for i in range(0, len(image_files), 2):
                pair_files = image_files[i:i+2]
                if len(pair_files) < 2:
                    print(f"警告: 文件夹 {folder_name} 中的图像数量为奇数，跳过最后一个不完整的对")
                    continue

                image_paths = [os.path.join(subject_folder_path, f) for f in pair_files]
                pair_index = i // 2
                prompt = prompt_groups[pair_index] if pair_index < len(prompt_groups) else prompt_groups[-1]
                generate_and_save_image(image_paths, folder_name, output_json_path, prompt, model, client, output_image_folder, pair_index, pair_files, new_ids_set, overall_progress)
                print(f"已处理图像对: {pair_files[0]} 和 {pair_files[1]}，文件夹: {folder_name}")
    
    save_new_ids_to_txt(new_ids_set, new_ids_output_path)

process_images_from_folder(folder_path, output_json_path, prompt_groups, model, client, output_image_folder)

# 多进程

In [3]:
import base64
import requests
import os
import json
import openai
import re
from tqdm import tqdm
from collections import defaultdict
from multiprocessing import Pool, Manager, cpu_count
import logging

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(processName)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)



# 路径配置
folder_path = "/data_lg/keru/DP_conclusion/DP_gpt_250502/valid_result"
output_json_path = "/data_lg/keru/DP_conclusion/DP_gpt_250617/output_log.json"
output_image_folder = "/data_lg/keru/DP_conclusion/DP_gpt_250617/generated_images"
error_txt_path = "/data_lg/keru/DP_conclusion/txt/result_error_2.txt"

# ID文本文件路径
valid_ids_path = "/data_lg/keru/DP_conclusion/txt/valid.txt"
existing_ids_1 = "/data_lg/keru/DP_conclusion/txt/250608.txt"
existing_ids_2 = "/data_lg/keru/DP_conclusion/txt/250502.txt"
new_ids_output_path = "/data_lg/keru/DP_conclusion/txt/250617.txt"

os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(os.path.dirname(new_ids_output_path), exist_ok=True)

# 提示词配置
prompt_groups = [
    "这是一件上装的前后，请参照为我重新生成上装的前后面,并将结果拼接呈现 ,高清,还原输入的衣服上所有花纹（碎花、文字、图案等）、颜色、样式。假如输入的衣服上面没有花纹，请不要额外添加！严格按照输入图的样式！背景为白色,细节清晰,图片规格,高1024,宽768*2（上装前面宽为768，上装后面为768），生成结果尽量保持居中，衣服长宽比需合理。后背中央空缺部分可能是头发遮挡，可按照空缺部分最近的颜色进行填充，生成完整衣服。一定要注意前后图案的生成要准确，尤其是后背！！！若有帽子、花纹、图案等务必还原！",
    "这是一件下装的前后，请参照为我重新生成下装的前后面 并将结果拼接呈现 ,高清,还原花纹（碎花、文字、图案等）、颜色、样式。假如输入的衣服上面没有花纹，请不要额外添加！严格按照输入图的样式！背景为白色,细节清晰,图片规格,高1024,宽768*2（上装前面宽为768，上装后面为768），生成结果尽量保持居中，衣服长宽比需合理。请注意，下装的前后长度需保持一致且合理，不会出现三角裤的情况，请避免内裤类型的生成,下装不包括鞋子和袜子！！裤子请按合理比例生成，避免长裤变成七分裤的情况。生成结果尽量真实",
    "这是一件套装的前后，套装可能是裙子，背带裤，请将他看作一件整体的衣服。请参照输入为我重新生成套装的前后面 并将结果拼接呈现 ,高清,还原花纹（碎花、文字、图案等）、颜色、样式。假如输入的衣服上面没有花纹，请不要额外添加！严格按照输入图的样式！背景为白色,细节清晰,图片规格,高1024,宽768*2（上装前面宽为768，上装后面为768），生成结果尽量保持居中，衣服长宽比需合理。后背中央空缺部分可能是头发遮挡，可按照空缺部分最近的颜色进行填充，生成完整衣服，生成结果尽量真实"
]

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def download_image(image_url, save_path):
    try:
        image_url = image_url.strip().rstrip(')')
        url_match = re.search(r'(https?://[^\s]+\.(?:png|jpg|jpeg|gif|bmp|webp))', image_url, re.IGNORECASE)
        if not url_match:
            logger.error(f"Invalid URL format: {image_url}")
            return False
        image_url = url_match.group(1)
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(image_url, headers=headers, timeout=30)
        if response.status_code == 200:
            with open(save_path, 'wb') as img_file:
                img_file.write(response.content)
            logger.info(f"Image saved to {save_path}")
            return True
        else:
            logger.error(f"Download failed ({response.status_code}): {image_url}")
            return False
    except Exception as e:
        logger.error(f"Download error: {e}")
        return False

def generate_and_save_image(worker_id, image_paths, folder_name, prompt, model, output_image_folder, pair_index, image_files):
    """工作进程处理单个图像对的函数"""
    # 为每个工作进程创建单独的OpenAI客户端
    client = openai.OpenAI(
        base_url=base_url,
        api_key=api_key,
    )
    
    base64_images = [encode_image(image_path) for image_path in image_paths]
    numbers = [re.search(r'part_(\d+)', image_file).group(1) for image_file in image_files]
    image_filename = f"result_{numbers[0]}_{numbers[1]}_generated.jpg"
    save_image_path = os.path.join(output_image_folder, folder_name, image_filename)
    os.makedirs(os.path.dirname(save_image_path), exist_ok=True)

    configure = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_images[0]}"}}
            ]}
        ]
    }

    try:
        response = client.chat.completions.create(**configure)
        data = response.choices[0].message.content
        success = download_image(data, save_image_path)
        
        output_log = {"worker_id": worker_id, "image_paths": image_paths, "response": data, "success": success}
        return output_log
    except Exception as e:
        logger.error(f"Error processing {image_files}: {e}")
        return {"worker_id": worker_id, "image_paths": image_paths, "error": str(e), "success": False}

def process_folder(folder_name, folder_path, output_json_path, prompt_groups, model, output_image_folder, error_queue, progress_queue):
    """处理单个文件夹的函数"""
    worker_id = os.getpid()  # 使用进程ID作为worker_id
    subject_folder_path = os.path.join(folder_path, folder_name)
    logger.info(f"Worker {worker_id}: 处理文件夹 {folder_name}")
    
    try:
        image_files = [f for f in os.listdir(subject_folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        image_files.sort(key=lambda x: int(re.search(r'part_(\d+)', x).group(1)))
        
        folder_output_logs = []
        
        for i in range(0, len(image_files), 2):
            pair_files = image_files[i:i+2]
            if len(pair_files) < 2:
                logger.warning(f"Worker {worker_id}: 文件夹 {folder_name} 中的图像数量为奇数，跳过最后一个不完整的对")
                continue

            image_paths = [os.path.join(subject_folder_path, f) for f in pair_files]
            pair_index = i // 2
            prompt = prompt_groups[pair_index] if pair_index < len(prompt_groups) else prompt_groups[-1]
            
            output_log = generate_and_save_image(
                worker_id, image_paths, folder_name, prompt, model, 
                output_image_folder, pair_index, pair_files
            )
            
            folder_output_logs.append(output_log)
            progress_queue.put(1)  # 更新进度
            
            if not output_log.get("success", False):
                error_queue.put(folder_name)
        
        # 线程安全地写入JSON日志
        with open(output_json_path, 'a') as json_file:
            for log in folder_output_logs:
                json.dump(log, json_file, indent=4)
                json_file.write("\n")
        
        return folder_name
    except Exception as e:
        logger.error(f"Worker {worker_id}: 处理文件夹 {folder_name} 时出错: {e}")
        error_queue.put(folder_name)
        return None

def load_ids_from_txt(file_path):
    if not os.path.exists(file_path):
        logger.warning(f"文件不存在: {file_path}，返回空集")
        return set()
    with open(file_path, 'r') as f:
        return {line.strip() for line in f if line.strip()}

def load_error_ids(error_txt_path):
    if not os.path.exists(error_txt_path):
        logger.warning(f"错误文件不存在: {error_txt_path}，返回空集")
        return set()
    with open(error_txt_path, "r") as f:
        return {line.strip() for line in f if line.strip()}

def save_new_ids_to_txt(new_ids_set, output_path):
    with open(output_path, 'w') as f:
        for folder_id in sorted(new_ids_set):
            f.write(f"{folder_id}\n")
    logger.info(f"已将 {len(new_ids_set)} 个新ID保存到: {output_path}")

def process_images_from_folder(folder_path, output_json_path, prompt_groups, model, output_image_folder):
    """主函数：使用多进程处理图像文件夹"""
    valid_ids = load_ids_from_txt(valid_ids_path)
    existing_ids = load_ids_from_txt(existing_ids_1) | load_ids_from_txt(existing_ids_2)
    error_ids = load_error_ids(error_txt_path)
    
    final_subjects = (valid_ids - existing_ids) | error_ids
    all_input_ids = {f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))}
    final_subjects = final_subjects & all_input_ids
    
    # 计算总任务数
    total_pairs = sum(
        (len([f for f in os.listdir(os.path.join(folder_path, f)) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]) + 1) // 2
        for f in final_subjects
    )
    
    logger.info(f"总共有 {len(final_subjects)} 个ID，预计处理 {total_pairs} 组图像对")
    
    # 创建进程间通信管理器
    with Manager() as manager:
        error_queue = manager.Queue()  # 用于收集错误的文件夹ID
        progress_queue = manager.Queue()  # 用于更新进度
        
        # 创建并启动进程池
        max_workers = min(cpu_count(), len(final_subjects))
        logger.info(f"使用 {max_workers} 个工作进程")
        
        with Pool(processes=max_workers) as pool:
            # 准备任务参数
            tasks = []
            for folder_name in sorted(final_subjects):
                tasks.append((
                    folder_name, folder_path, output_json_path, 
                    prompt_groups, model, output_image_folder,
                    error_queue, progress_queue
                ))
            
            # 创建总进度条
            with tqdm(total=total_pairs, desc="整体进度", unit="组") as overall_progress:
                # 异步执行任务
                results = [pool.apply_async(process_folder, args=task) for task in tasks]
                
                # 处理进度更新
                completed = 0
                while completed < total_pairs:
                    try:
                        progress_queue.get(timeout=1)
                        completed += 1
                        overall_progress.update(1)
                    except Exception:
                        pass
                
                # 获取所有结果
                processed_folders = [result.get() for result in results]
        
        # 收集错误ID
        new_error_ids = set()
        while not error_queue.empty():
            new_error_ids.add(error_queue.get())
        
        # 保存新处理的ID
        new_ids_set = set(f for f in processed_folders if f is not None)
        save_new_ids_to_txt(new_ids_set, new_ids_output_path)
        
        # 更新错误ID文件
        if new_error_ids:
            with open(error_txt_path, 'a') as f:
                for folder_id in new_error_ids:
                    f.write(f"{folder_id}\n")
            logger.warning(f"已将 {len(new_error_ids)} 个错误ID添加到 {error_txt_path}")

if __name__ == "__main__":
    # 清空输出JSON文件（如果存在）
    if os.path.exists(output_json_path):
        os.remove(output_json_path)
    
    process_images_from_folder(folder_path, output_json_path, prompt_groups, model, output_image_folder)

2025-06-18 16:57:52,534 - MainProcess - INFO - 总共有 4510 个ID，预计处理 8448 组图像对
2025-06-18 16:57:52,696 - MainProcess - INFO - 使用 128 个工作进程
整体进度:   0%|          | 0/8448 [00:00<?, ?组/s]2025-06-18 16:57:53,461 - ForkPoolWorker-2 - INFO - Worker 1618488: 处理文件夹 102025
2025-06-18 16:57:53,471 - ForkPoolWorker-4 - INFO - Worker 1618493: 处理文件夹 102027
2025-06-18 16:57:53,465 - ForkPoolWorker-3 - INFO - Worker 1618491: 处理文件夹 102026
2025-06-18 16:57:53,488 - ForkPoolWorker-5 - INFO - Worker 1618496: 处理文件夹 102029
2025-06-18 16:57:53,500 - ForkPoolWorker-17 - INFO - Worker 1618556: 处理文件夹 102044
2025-06-18 16:57:53,504 - ForkPoolWorker-7 - INFO - Worker 1618502: 处理文件夹 102031
2025-06-18 16:57:53,506 - ForkPoolWorker-8 - INFO - Worker 1618505: 处理文件夹 102032
2025-06-18 16:57:53,511 - ForkPoolWorker-6 - INFO - Worker 1618499: 处理文件夹 102030
2025-06-18 16:57:53,515 - ForkPoolWorker-9 - INFO - Worker 1618509: 处理文件夹 102033
2025-06-18 16:57:53,525 - ForkPoolWorker-11 - INFO - Worker 1618517: 处理文件夹 102035
2025-06-